In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords 
import sklearn.ensemble as ens
import sklearn.feature_extraction.text as fet
from bs4 import BeautifulSoup
import re

#загрузка датасетов
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3)

#преобразование текста рецензий к единому виду, регистру, а также удаление ссылок и пунктуации
def pre_process_reviews(review):
    obj = BeautifulSoup(review).get_text()   
    process = re.sub("[^a-zA-Z]", " ", obj) 
    words = process.lower().split()                             
    stops = set(stopwords.words("english"))                  
    mean_words = [w for w in words if not w in stops]   
    return(" ".join(mean_words))

#векторизация предложений
train['review']=[pre_process_reviews(review) for review in train['review']]
clean_train_reviews = []
for i in range(0, train["review"].size):
    clean_train_reviews.append( pre_process_reviews( train["review"][i] ))
vector = fet.CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 25000) 
train_data_features = vector.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

#применяем случайный лес
rfc = ens.RandomForestClassifier(n_estimators = 250) 
rfc = rfc.fit(train_data_features, train["sentiment"])
clean_test_reviews = []
for i in range(0,len(test["review"])):
    clean_review = pre_process_reviews(test["review"][i])
    clean_test_reviews.append(clean_review)
test_data_features = vector.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()
result = rfc.predict(test_data_features)

#вывод результата
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv("submission.csv", index=False, quoting=3)